In [81]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [82]:
time_slots = []
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
hours = range(7, 23)
for day in days:
    for hour in hours:
        time_slot = f'{day}_{hour}'
        time_slots.append(time_slot)

In [83]:
activity_times = {
    'robotics':['Mon_15','Mon_16','Mon_17','Wed_15','Wed_16','Thu_15','Thu_16','Thu_17'],
    'track':['Mon_15','Mon_16','Tue_15','Tue_16','Wed_15','Wed_16','Thu_15','Thu_16','Fri_15','Fri_16','Sat_9','Sat_10'],
    'swim':['Mon_19','Mon_20','Tue_16','Tue_17','Tue_18','Wed_18','Wed_19','Wed_20','Fri_16','Fri_17','Fri_18','Sat_7','Sat_8','Sat_9','Sun_17','Sun_18','Sun_19'],
    'tutoring':['Thu_17'],
    'music':time_slots,
}

In [84]:
activity_vars = {}
for activity in activity_times:
    for time in activity_times[activity]:
        act_var = model.NewBoolVar(f'{activity}_{time}')
        activity_vars[(activity,time)] = act_var

In [85]:
for slot in time_slots:
    relevant_vars = [activity_vars[(act, slot)] for act in activity_times if (act, slot) in activity_vars]
    model.Add(sum(relevant_vars) <= 1)

In [86]:
target_hours = {'robotics': 7, 'track': 11, 'swim': 16, 'tutoring': 1, 'music': 2}
for act, target in target_hours.items():
    vars_for_activity = [activity_vars[(act,slot)] for slot in activity_times[act]]
    model.Add(sum(vars_for_activity)>= target-2)
    model.Add(sum(vars_for_activity)<=target+1)

In [87]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for slot in time_slots:
        for act in activity_times:
            if (act, slot) in activity_vars and solver.Value(activity_vars[(act,slot)]) == 1:
                print(f'{slot}: {act}')


In [88]:
for act in target_hours:
    hours = sum(
        solver.Value(activity_vars[(act,slot)])
        for slot in activity_times[act]
        if (act,slot) in activity_vars
    )
    print(f'{act}: {hours} hours total')


robotics: 14578801267165440 hours total
track: 143311116902450 hours total
swim: 9888368243356774991 hours total
tutoring: 140724125030154 hours total
music: -9368862241289813310 hours total
